In [6]:
import pandas as pd
import numpy as np

In [7]:
df_input = """
DocID       0   1    2   3   4    5

CAT123     CAT  1   12 123  123  123
DOG14567   DOG  1   14 145 1456 14567
BIRD32     BIRD 3   32  32   32   32
"""

In [8]:
def SOF_to_df(df_input):
    """
    Args:
        df_input (string): output of print(pd.DataFrame)
    Return:
        df (pd.DataFrame)
    """
    import pandas as pd

    raw = [s.split(' ') for s in df_input.split('\n')]
    cleaned = []
    for section in raw:
        if section != ['']:
            cleaned.append([num for num in section if num != ''])

    df = pd.DataFrame(cleaned)
    df.columns = df.loc[0].tolist()
    return df.drop(0, axis=0)

In [52]:
df = SOF_to_df(df_input)

In [53]:
df.loc[:,'1':'5'] = df.loc[:,'1':'5'].apply(pd.to_numeric)

In [54]:
for i in [5, 4, 3, 2]:
    df.iloc[:,i+1][df.iloc[:, i + 1] - df.iloc[:, i] == 0] = np.nan

Checking other solutions

In [78]:
df = SOF_to_df(df_input)
df.loc[:,'1':'5'] = df.loc[:,'1':'5'].apply(pd.to_numeric)

In [79]:
df['1']

1    1
2    1
3    3
Name: 1, dtype: object

In [80]:
df['1'].duplicated()

1    False
2     True
3    False
Name: 1, dtype: bool

In [68]:
df

,DocID,0,1,2,3,4,5
1,CAT123,CAT,1,12,123,123,123
2,DOG14567,DOG,1,14,145,1456,14567
3,BIRD32,BIRD,3,32,32,32,32


In [69]:
df.apply(pd.Series.duplicated, 1)

,DocID,0,1,2,3,4,5
1,False,False,False,False,False,True,True
2,False,False,False,False,False,False,False
3,False,False,False,False,True,True,True


In [71]:
help(df.mask)

Help on method mask in module pandas.core.generic:

mask(self, cond, other=nan, inplace=False, axis=None, level=None, errors='raise', try_cast=False, raise_on_error=None) method of pandas.core.frame.DataFrame instance
    Return an object of same shape as self and whose corresponding
    entries are from self where `cond` is False and otherwise are from
    `other`.
    
    Parameters
    ----------
    cond : boolean NDFrame, array-like, or callable
        Where `cond` is False, keep the original value. Where
        True, replace with corresponding value from `other`.
        If `cond` is callable, it is computed on the NDFrame and
        should return boolean NDFrame or array. The callable must
        not change input NDFrame (though pandas doesn't check it).
    
        .. versionadded:: 0.18.1
            A callable can be used as cond.
    
    other : scalar, NDFrame, or callable
        Entries where `cond` is True are replaced with
        corresponding value from `other`

In [83]:
df.mask(df < 2)

,DocID,0,1,2,3,4,5
1,CAT123,CAT,NaN,12,123,123,123
2,DOG14567,DOG,NaN,14,145,1456,14567
3,BIRD32,BIRD,3,32,32,32,32


In [85]:
df.where(df <2)

,DocID,0,1,2,3,4,5
1,NaN,NaN,1,NaN,NaN,NaN,NaN
2,NaN,NaN,1,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


so looks like mask is the opposite of where, just replace NaN in the other places

In [86]:
df.mask(df.apply(pd.Series.duplicated, 1))

,DocID,0,1,2,3,4,5
1,CAT123,CAT,1,12,123,NaN,NaN
2,DOG14567,DOG,1,14,145,1456,14567
3,BIRD32,BIRD,3,32,NaN,NaN,NaN


this solution is pretty smart